https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce#scrollTo=8DKMc0fiej4N

# Настройка доступов и подгрузка библиотек

In [ ]:
meme_pattern = 'waiting skeleton'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -s "/content/drive/My Drive" /gdrive

In [ ]:
%tensorflow_version 1.x
!pip install git+https://github.com/SerSmith/gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
  Cloning https://github.com/SerSmith/gpt-2-simple to /tmp/pip-req-build-fybvb7og
  Running command git clone -q https://github.com/SerSmith/gpt-2-simple /tmp/pip-req-build-fybvb7og
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=25644 sha256=891a8d726b2a18d0dac0cfe1b0acf4d2de6c9b8f2093a5a5e7d878bc612889b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-2_ew3y07/wheels/05/e5/9d/9b6ebf15d32922867291a66ba17b8610a5621e6cd1a399fba5
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Подготовка датасета для дообучения

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import os
import pickle
import nltk
import shutil
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

path_raw_data = '/gdrive/MADE_PROJECT/raw_data/memes_all_eng_v2.csv'
path_folder_gdisk = '/content/drive/My Drive'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv(path_raw_data)
df['template_name_prep'].value_counts()

batman slapping robin    18241
x x everywhere           16677
waiting skeleton         12314
change my mind           12169
disaster girl             9418
Name: template_name_prep, dtype: int64

In [ ]:
def prepare_text(text):
  text = str(text).strip().upper()
  text = text.replace(r"\s", " ")
  text = text.replace(r"\n", " ")
  text = text.replace(r"&", "and")
  text = text.rstrip('|')
  return text

def prepare_train(data, template, dest_path, min_size=10):
  out = df[df['template_name_prep'] == template]['text'].drop_duplicates().reset_index(drop=True).apply(prepare_text)
  out = out[out.apply(len) >= min_size ]
  print(out.shape)
  out.to_csv(dest_path, index=False, header=False)
  return out

q = prepare_train(df, meme_pattern, os.path.join(path_folder_gdisk, f'{meme_pattern}.csv'))
table = os.path.join(path_folder_gdisk, f'{meme_pattern}.csv')
!cat "$table" | head -n 10

(12220,)
WAITING|FOR THE BEAT TO DROP
ME WAITING ON|A BBC CORONA MEME
"TEACHERS WHEN THEY SAY|""IMMA WAIT TILL IT QUIET"""
|FINDING OUT MY LIKE GREAT-GRANDMA GOT MARRIED AT 14 TO A 50 YEAR OLD MAN
PEOPLE WAITING FOR|THE LOCKDOWN TO END
|JUST WAITING FOR CAM TO LOAD
ME WAITING FOR SOME SORT OF GUIDANCE
|WAITING FOR SCHOOL TO REOPEN BE LIKE
I'VE BEEN WAITING FOREVER!
ADA COUNTY EMPLOYEES WAITING|FOR SURFACE PRO TO RESTART


Запуск дообучения

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
!nvidia-smi

Sat Nov 28 15:08:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.mount_gdrive()
gpt2.download_gpt2(model_name="124M")
gpt2.copy_file_from_gdrive(f'{meme_pattern}.csv')

history = gpt2.finetune(sess,
        f'{meme_pattern}.csv',
        steps=2000,
        model_name='124M',
        model_dir='models',
        combine=50000,
        batch_size=1,
        learning_rate=0.0001,
        accumulate_gradients=5,
        restore_from='latest',
        run_name=meme_pattern,
        checkpoint_dir='checkpoint',
        sample_every=50,
        sample_length=50,
        sample_num=10,
        multi_gpu=False,
        save_every=1000,
        print_every=50)

Fetching checkpoint: 1.05Mit [00:00, 295Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 134Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 435Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:   0%|                          | 0.00/498M [00:00<?, ?it/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 200Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 461Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 155Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 197Mit/s]                                                       


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 61.79it/s]

...

[1900 | 4724.28] loss=0.05 avg=0.44
 SHOW WAITING AND HE DIDN'T SAY "OK THANK YOU" AND LEFT.<|endoftext|>
<|startoftext|>IN LINE FOR LUNCH LIKE. ''<|endoftext|>
<


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[1950 | 4848.51] loss=0.05 avg=0.43
|>
<|startoftext|>|MY MOM WAITING FOR HER DATE<|endoftext|>
<|startoftext|>ME WAITING|FOR A NEW LOYAL CHICK<|


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[2000 | 4972.87] loss=0.04 avg=0.41
Saving checkpoint/waiting skeleton/model-2000
Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
with open(f'/gdrive/{meme_pattern}_history.pickle', 'wb') as f:
  pickle.dump(history, f)

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=meme_pattern)

# Генерация текста

In [ ]:
# sess = gpt2.start_tf_sess()
# gpt2.load_gpt2(sess, run_name='disaster girl 12 11')
# gpt2.generate(sess,
#               length=100,
#               temperature=0.7,
#               prefix="Change my mind",
#               nsamples=5,
#               batch_size=5,
#               truncate='EOF'
#               )